# DNA extraction of  96-well cultures with magnetic beads

## Protocol Initialization
Ensure you have calibrated the pipettes before doing this protocol

In [ ]:
from opentrons import execute, simulate
import json
import time

Use one of the code lines below for simulation or execution mode:

**Simulation mode**
```
protocol = simulate.get_protocol_api('2.13')
```



**Execution mode**
```
protocol = execute.get_protocol_api('2.13')
```

In [ ]:
protocol = simulate.get_protocol_api('2.12')

In [ ]:
protocol.home()

## Declaring Labware
- **Right mount**: An 8-channel p300 pipette
- **Slot 4**: 12-well reservoir with buffers (fill up to volume for required samples: max 96 samples)
    - Well 1: Lysis buffer (Up to 12 mL)
    - Well 2, 3: Neutralization buffer (Up to 15 mL each well)
    - Well 4, 5: Wash buffer (Up to 15 mL each well)
    - Well 6, 7: Ethanol (Up to 12 mL each well)
    - Well 8: Elution Buffer (Up to 10mL)
- **Slot 7**: 96-deepwell plate (square wells) with ressuspended bacterial cell pellets (in 120 uL ressuspension buffer)
- **Slot 9**: Output plate in 96-well format
- **Slot 10**: Magnetic module with 96-deepwell plate (square wells) with 100uL of magnetic beads
- **Slot 11**: 1-well reservoir for buffer waste
- **Slots 1, 2, 3, 5, 6**: Tipracks for p300

Replace with appropriate labware codes in the cell below if needed
If you need to use custom labware, use our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [ ]:
# Set labware
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    plate_96 = protocol.load_labware_from_definition(labware_def, 7)

plate_out = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '9')

waste = protocol.load_labware('nest_1_reservoir_195ml', '11')

with open ('custom_labware/enzymax_12_reservoir_20ml.json') as labware_file:
    labware_def = json.load(labware_file)
    liquids_reservoir = protocol.load_labware_from_definition(labware_def, 4)
    
tipracks_300 = [protocol.load_labware('opentrons_96_tiprack_300ul', '1'),
               protocol.load_labware('opentrons_96_tiprack_300ul', '2'),
               protocol.load_labware('opentrons_96_tiprack_300ul', '3'),
               protocol.load_labware('opentrons_96_tiprack_300ul', '5'),
               protocol.load_labware('opentrons_96_tiprack_300ul', '6')]

p300 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=tipracks_300)

mag_module = protocol.load_module("magnetic module", '10')
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    mag_plate = mag_module.load_labware_from_definition(labware_def, 10)

# Define the number of samples here
- Run the next cell and fill out the prompts
- You can use from 1 to 96 samples.

In [ ]:
sample_no = 16


col_no = 1+(sample_no-1)//8
columns = plate_96.rows()[0][:col_no]
lysis_buff = liquids_reservoir['A1']
neut_buff = [liquids_reservoir['A2'], liquids_reservoir['A3']]
wash_buff = [liquids_reservoir['A4'], liquids_reservoir['A5']]
ethanol = [liquids_reservoir['A6'], liquids_reservoir['A7']]
elut_buff = liquids_reservoir['A8']

## Labware Calibration

Change the x, y, and z variables (**in mm**) in the "set_offset" functions and run each cell repeatedly until the positioning is correct for that piece of labware. You will have to use a "dummy" plate to calibrate the labware that is not on the deck yet (competent cells).

### Tipracks:

In [ ]:
protocol.comment('-- CALIBRATION --')

In [ ]:
# Calibrate tiprack for p300. Re-run calibration cell with different numbers if needed
tipracks_300[0].set_offset(x=0.7, y=0.4, z=0.)
p300.move_to(tipracks_300[0]['A1'].top())

In [ ]:
# Calibrate tiprack for p300. Re-run calibration cell with different numbers if needed
tipracks_300[1].set_offset(x=0.7, y=0.6, z=0.)
p300.move_to(tipracks_300[1]['A1'].top())

### Plates/Racks:

In [ ]:
p300.pick_up_tip()

In [ ]:
# Calibrate remaining labware. Re-run calibration cells with different numbers if needed
plate_96.set_offset(x=0.0, y=0.5, z=0.7)
p300.move_to(plate_96['A1'].top())

In [ ]:
liquids_reservoir.set_offset(x=0.0, y=0.0, z=0.0)
p300.move_to(liquids_reservoir['A1'].top())

In [ ]:
mag_plate.set_offset(x=0.0, y=0.9, z=0.7)
p300.move_to(mag_plate['A1'].top())

In [ ]:
plate_out.set_offset(x=0.5, y=0.5, z=0.3)
p300.move_to(plate_out['A1'].top())

In [ ]:
waste.set_offset(x=0.0, y=0.0, z=0.0)
p300.move_to(waste['A1'].top())

In [ ]:
p300.return_tip()
p300.reset_tipracks()

## Protocol Execution
### Add lysis and neutralization buffer
- Centrifuge at max speed for 20 minutes when done

In [ ]:
times = []
protocol.comment('-- ADDING LYSIS BUFFER --')
p300.pick_up_tip()
for col in columns:
    p300.aspirate(120, lysis_buff)
    p300.dispense(120, col.top())
    p300.blow_out()
    times.append({'column':col, 'time':time.time()})
p300.drop_tip()

remaining = 300 - (time.time() - times[0]['time'])
print(remaining, 'seconds wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(seconds = remaining)

protocol.comment('-- ADDING NEUTRALIZATION BUFFER --')

buffer_left = 15000
buff_well = 0

p300.pick_up_tip()
for remain in times:
    col = remain['column']
    p300.aspirate(240, neut_buff[buff_well])
    p300.dispense(240, col.top())
    p300.blow_out()
    
    buffer_left -= 240 * 8
    if buffer_left <= 240 * 8:
        buff_well += 1
        buffer_left = 15000
p300.drop_tip()

### Add supernatant and ethanol to beads
- Aspirate slowly and a bit higher to avoid cell debris
- After this step, incubate at room temperature for 20 minutes, vortexing briefly after every 4 minutes 

In [ ]:
protocol.comment('-- ADDING SAMPLES AND EHTANOL TO MAG PLATE --')

buffer_left = 12000
buff_well = 0

p300.pick_up_tip()
for col in columns:
    p300.aspirate(200, ethanol[buff_well])
    p300.dispense(200, mag_plate[col.well_name].top())
    p300.blow_out()
    
    buffer_left -= 200 * 8
    if buffer_left <= 200 * 8:
        buff_well += 1
        buffer_left = 12000
p300.drop_tip()

for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, col.bottom(z=5.0), rate=0.2)
    p300.dispense(300, mag_plate[col.well_name])
    p300.mix(5, 250, mag_plate[col.well_name], rate=3.0)
    p300.drop_tip()

### Remove supernatant from beads

In [ ]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes = 2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, mag_plate[col.well_name].bottom(z=2.0), rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.aspirate(300, mag_plate[col.well_name].bottom(z=2.0), rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.drop_tip()
mag_module.disengage()

### Add wash buffer
- After this step, incubate at room temperature for 10 minutes, vortexing briefly after every 2 minutes

In [ ]:
buffer_left = 15000
buff_well = 0

p300.pick_up_tip()
for col in columns:
    p300.aspirate(300, wash_buff[buff_well])
    p300.dispense(300, mag_plate[col.well_name].top())
    p300.blow_out()
    
    buffer_left -= 300 * 8
    if buffer_left <= 300 * 8:
        buff_well += 1
        buffer_left = 15000
p300.drop_tip()

### Remove wash buffer
- After this step, dry plate at 65°C for 20 minutes

In [ ]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes = 2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(300, mag_plate[col.well_name].bottom(z=2.0), rate=0.2)
    p300.dispense(300, waste['A1'].top())
    p300.blow_out(waste['A1'].top())
    p300.drop_tip()
mag_module.disengage()

### Add elution buffer
- After this step, incubate at room temperature for 10 minutes, vortexing briefly every 2 minutes

In [ ]:
p300.pick_up_tip()
for col in columns:
    p300.aspirate(100, elut_buff)
    p300.dispense(100, mag_plate[col.well_name].top())
    p300.blow_out()
p300.drop_tip()

### Transfer elution to output plate

In [ ]:
mag_module.engage(12)
print('2 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes=2)

# Pick up supernatant slowly to avoid getting beads
for col in columns:
    p300.pick_up_tip()
    p300.aspirate(100, mag_plate[col.well_name].bottom(z=2.0), rate=0.2)
    p300.dispense(100, plate_out[col.well_name])
    p300.drop_tip()

mag_module.disengage()

In [ ]:
protocol.home()

In [ ]:
protocol.commands()